In [35]:
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, ParameterGrid

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


from utils import calcular_estatisticas, imprimir_estatisticas, rejeitar_hip_nula
from joblib import delayed, Parallel

from tqdm.notebook import tqdm

In [32]:
df = pd.read_csv('heart_disease.csv')
df.columns = [i.strip() for i in df.columns]
X = df.drop('Disease', axis=1)
y = df['Disease'].values.ravel()
df

,age,sex,chest pain type,resting blood pressure,serum cholestoral in mg/dl,fasting blood sugar > 120 mg/dl,resting electrocardiographic results,maximum heart rate achieved,exercise induced angina,oldpeak,slope of peak,number of major vessels,thal,Disease
0,70,1,4,130,322,0,2,109,0,2.4,2,3,3,1
1,67,0,3,115,564,0,2,160,0,1.6,2,0,7,0
2,57,1,2,124,261,0,0,141,0,0.3,1,0,7,1
3,64,1,4,128,263,0,0,105,1,0.2,2,1,7,0
4,74,0,2,120,269,0,2,121,1,0.2,1,1,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,52,1,3,172,199,1,0,162,0,0.5,1,0,7,0
266,44,1,2,120,263,0,0,173,0,0.0,1,0,7,0
267,56,0,2,140,294,0,2,153,0,1.3,2,0,3,0
268,57,1,4,140,192,0,0,148,0,0.4,2,0,6,0


In [33]:
transformers = [
    ("sex", OneHotEncoder(), ['sex']),
    ("fasting blood sugar > 120 mg/dl", OneHotEncoder(), ['fasting blood sugar > 120 mg/dl']),
    ("exercise induced angina", OneHotEncoder(), ['exercise induced angina']),
    ("resting electrocardiographic results", OneHotEncoder(), ['resting electrocardiographic results']),
    ("chest pain type", OneHotEncoder(), ['chest pain type']),
]

ct = ColumnTransformer(transformers, sparse_threshold=0, remainder='passthrough')
ct.fit(X)
X_ohe = ct.transform(X)
X_ohe = pd.DataFrame(data=X_ohe, columns = ct.get_feature_names())
X_ohe

,sex__x0_0,sex__x0_1,fasting blood sugar > 120 mg/dl__x0_0,fasting blood sugar > 120 mg/dl__x0_1,exercise induced angina__x0_0,exercise induced angina__x0_1,resting electrocardiographic results__x0_0,resting electrocardiographic results__x0_1,resting electrocardiographic results__x0_2,chest pain type__x0_1,...,chest pain type__x0_3,chest pain type__x0_4,age,resting blood pressure,serum cholestoral in mg/dl,maximum heart rate achieved,oldpeak,slope of peak,number of major vessels,thal
0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,70.0,130.0,322.0,109.0,2.4,2.0,3.0,3.0
1,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,67.0,115.0,564.0,160.0,1.6,2.0,0.0,7.0
2,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,57.0,124.0,261.0,141.0,0.3,1.0,0.0,7.0
3,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,1.0,64.0,128.0,263.0,105.0,0.2,2.0,1.0,7.0
4,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,74.0,120.0,269.0,121.0,0.2,1.0,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,52.0,172.0,199.0,162.0,0.5,1.0,0.0,7.0
266,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,44.0,120.0,263.0,173.0,0.0,1.0,0.0,7.0
267,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,56.0,140.0,294.0,153.0,1.3,2.0,0.0,3.0
268,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,57.0,140.0,192.0,148.0,0.4,2.0,0.0,6.0


In [34]:
from sklearn.preprocessing import StandardScaler

def selecionar_melhor_modelo(classificador, X_treino, X_val, y_treino, y_val, n_jobs=4, 
                             cv_folds=None, params={}):
    
    def treinar_ad(X_treino, X_val, y_treino, y_val, params):
        clf = classificador(**params)
        clf.fit(X_treino, y_treino)
        pred = clf.predict(X_val)
        
        if len(set(y_treino)) > 2:
            return f1_score(y_val, pred, average='weighted')
        else:
            return f1_score(y_val, pred)
    
    
    if cv_folds is not None:
        #Se for pra usar validação cruzada, usar GridSearchCV
        score_fn = 'f1' if len(set(y_treino)) < 3 else 'f1_weighted'
        
        clf = GridSearchCV(classificador(), params, cv=cv_folds, n_jobs=n_jobs, scoring=score_fn)
        #Passar todos os dados (Treino e Validação) para realizar a seleção dos parâmetros.
        clf.fit(np.vstack((X_treino, X_val)), [*y_treino, *y_val])
        
        melhor_comb = clf.best_params_
        melhor_val = clf.best_score_
        
    else:
        param_grid = list(ParameterGrid(params))
        
        f1s_val = Parallel(n_jobs=n_jobs)(delayed(treinar_ad)
                                         (X_treino, X_val, y_treino, y_val, p) for p in param_grid)

        melhor_val = max(f1s_val)
        melhor_comb = param_grid[np.argmax(f1s_val)]
        
        clf = classificador(**melhor_comb)
        
        clf.fit(np.vstack((X_treino, X_val)), [*y_treino, *y_val])
    
    return clf, melhor_comb, melhor_val

def do_cv(classificador, X, y, cv_splits, param_cv_folds=None, n_jobs=8, scale=False, params={}):

    skf = StratifiedKFold(n_splits=cv_splits, shuffle=True, random_state=1)

    f1s = []
    
    pgb = tqdm(total=cv_splits, desc='Folds avaliados')
    
    for treino_idx, teste_idx in skf.split(X, y):

        X_treino = X[treino_idx]
        y_treino = y[treino_idx]

        X_teste = X[teste_idx]
        y_teste = y[teste_idx]

        X_treino, X_val, y_treino, y_val = train_test_split(X_treino, y_treino, stratify=y_treino, test_size=0.2, random_state=1)
        
        if scale:
            ss = StandardScaler()
            X_treino = ss.fit_transform(X_treino)
            X_teste = ss.transform(X_teste)
            X_val = ss.transform(X_val)        

        ad, melhor_comb, _ = selecionar_melhor_modelo(classificador, X_treino, X_val, y_treino, y_val, 
                                                      n_jobs=n_jobs, cv_folds=param_cv_folds, params=params)
        pred = ad.predict(X_teste)

        if len(set(y_treino)) > 2:
            f1 = f1_score(y_teste, pred, average='weighted')
        else:
            f1 = f1_score(y_teste, pred)
        f1s.append(f1)
        
        pgb.update(1)
        
    pgb.close()
    
    return f1s

In [39]:
f1_random_forest = do_cv(RandomForestClassifier, X_ohe.values, y, 10, 10, 8, False, params={'n_estimators': [100, 250, 500, 1000]})

Folds avaliados:   0%|          | 0/10 [00:00<?, ?it/s]

In [46]:
f1_svm = do_cv(SVC, X_ohe.values, y, 10, 10, n_jobs=8, scale=True, params={'C' : [1, 10, 100, 1000], 'gamma' : ['auto', 'scale', 2e-2, 2e-3, 2e-4]})


Folds avaliados:   0%|          | 0/10 [00:00<?, ?it/s]

In [43]:
f1_knn = do_cv(KNeighborsClassifier, X_ohe.values, y, 10, 10, n_jobs=8, scale=True, params={'n_neighbors' : range(1,30,2)})

Folds avaliados:   0%|          | 0/10 [00:00<?, ?it/s]

In [56]:
ad = do_cv(DecisionTreeClassifier, X_ohe.values, y, 10, None, 8, False ,params={})

Folds avaliados:   0%|          | 0/10 [00:00<?, ?it/s]

In [57]:
imprimir_estatisticas(f1_random_forest)
imprimir_estatisticas(f1_svm)
imprimir_estatisticas(f1_knn)
imprimir_estatisticas(ad)

Resultados: 0.79 +- 0.10, min: 0.67, max: 0.96
Resultados: 0.82 +- 0.09, min: 0.72, max: 0.96
Resultados: 0.79 +- 0.10, min: 0.63, max: 0.96
Resultados: 0.70 +- 0.06, min: 0.60, max: 0.78


In [60]:
rejeitar_hip_nula(ad, f1_random_forest)

(True, 0.026869031537963252)

In [61]:
rf_test = ('rf', RandomForestClassifier, False, {'criterion' : ['gini', 'entropy'], 'random_state' : [1]})
ad_def_test = ('ad_default', DecisionTreeClassifier, False, {'criterion' : ['gini', 'entropy'], 'random_state' : [1]})
ad_tuned_test = ('ad_tuned', DecisionTreeClassifier, False, {'min_samples_leaf' : [3, 5, 10, 15, 20, 30],
                                                 'min_samples_split' : [2, 4, 8, 16, 32, 40],
                                                 'max_depth' : [None, 2, 3, 4, 5],
                                                 'criterion' : ['gini', 'entropy'], 'random_state' : [1]})
svm_test = ('svm', SVC, True, {'C' : [1, 10, 100, 1000], 'gamma' : ['auto', 'scale']})
knn_test = ('knn', KNeighborsClassifier, True, {'n_neighbors' : range(1,30,2)})

tests = [rf_test, ad_def_test, ad_tuned_test, svm_test, knn_test]

In [62]:
resultados = {}
for nome, classificador, scale, params in tests:
    r = do_cv(classificador, X.values, y, 10, 10, 8, scale, params)
    resultados[nome] = r
    

Folds avaliados:   0%|          | 0/10 [00:00<?, ?it/s]

Folds avaliados:   0%|          | 0/10 [00:00<?, ?it/s]

Folds avaliados:   0%|          | 0/10 [00:00<?, ?it/s]

Folds avaliados:   0%|          | 0/10 [00:00<?, ?it/s]

Folds avaliados:   0%|          | 0/10 [00:00<?, ?it/s]

In [63]:
for teste in resultados:
    print(teste.rjust(10), end=' - ')
    imprimir_estatisticas(resultados[teste])

        rf - Resultados: 0.81 +- 0.10, min: 0.64, max: 0.96
ad_default - Resultados: 0.73 +- 0.08, min: 0.58, max: 0.88
  ad_tuned - Resultados: 0.79 +- 0.13, min: 0.56, max: 0.96
       svm - Resultados: 0.82 +- 0.10, min: 0.69, max: 0.96
       knn - Resultados: 0.80 +- 0.11, min: 0.64, max: 0.96


In [64]:
largura = max(map(len,resultados.keys()))+2
print(" " * largura , end="")
      
for t in resultados:
    print(t.center(largura), end='')
print()

for t in resultados:
    print(t.center(largura), end='')
    for t2 in resultados:
        d, p = rejeitar_hip_nula(resultados[t], resultados[t2], alpha=0.1)
        print(("%.02f%s" % (p, ' (*)' if d else '')).center(largura), end='')
    print()

                 rf      ad_default   ad_tuned      svm         knn     
     rf         1.00      0.07 (*)      0.68        0.84        0.86    
 ad_default   0.07 (*)      1.00        0.26      0.06 (*)      0.13    
  ad_tuned      0.68        0.26        1.00        0.57        0.81    
    svm         0.84      0.06 (*)      0.57        1.00        0.73    
    knn         0.86        0.13        0.81        0.73        1.00    
